In [1]:
# Importamos librerías necesarias
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

# Lista de URLs de las 49 páginas
webs = ["https://www.padelnuestro.com/palas-padel/?p={}".format(i) for i in range(1, 50)]

# Listas vacías para incluir la distinta información
palas = []
precio = []
marca = []
color = []
producto = []
balance = []
nucleo = []
cara = []
dureza = []
nivel_de_juego = []
acabado = []
forma = []
superficie = []
tipo_de_juego = []
jugador = []

# Definir palabras clave para cada categoría
keywords = {
    'Marca': [
        "Adidas", "Bullpadel", "Head", "Wilson", "Dunlop", "Drop Shot", "Softee", "Enebe", "Babolat", "Black Crown",
        "Cartri", "Eme", "Gama", "Joma", "Just Ten", "Kaitt", "Kelme", "Kuikma", "Nox", "Orygen", "Royal Padel",
        "Siux", "Star Vie", "Varlion", "Vibor-A", "Wingpadel", "Vision", "Rossignol", "DHS", "Tecnifibre",
        "Power Padel", "Oxdog", "Prokennex", "Prince", "Volkl", "Slazenger", "ShockOut", "RoyalPadel", "Qustom Padel",
        "Paddle Coach"
    ],
    'Color': ['rojo', 'azul', 'verde', 'amarillo', 'negro', 'blanco', 'naranja', 'rosa', 'morado', 'gris', 'marrón', 'plateado', 'dorado'],
    'Producto': ['pala', 'raqueta', 'padel bat'],
    'Balance': ['alto', 'medio', 'bajo'],
    'Nucleo': ['EVA', 'foam', 'híbrido'],
    'Cara': ['fibra de carbono', 'fibra de vidrio', 'graphene', 'kevlar'],
    'Dureza': ['dura', 'media', 'blanda'],
    'Nivel de Juego': ['principiante', 'intermedio', 'avanzado', 'profesional'],
    'Acabado': ['brillante', 'mate', 'rugoso'],
    'Forma': ['diamante', 'redonda', 'lágrima', 'híbrido'],
    'Superficie': ['lisa', 'rugosa'],
    'Tipo de Juego': ['control', 'potencia', 'polivalente'],
    'Jugador': ['hombre', 'mujer', 'junior']
}

# Función para encontrar el valor correspondiente a una categoría basada en las palabras clave
def encontrar_valor(attrs, label, keywords):
    for attr in attrs:
        text = attr.text.strip().lower()
        for keyword in keywords:
            if keyword.lower() in text:
                return text
    return "No data"

# Función para extraer detalles de cada producto
def extraer_detalles_producto(url):
    try:
        page = requests.get(url)
        page.raise_for_status()
    except requests.RequestException as e:
        print(f"Error al acceder a {url}: {e}")
        return

    soup = BeautifulSoup(page.content, 'html.parser')

    # Extraer el nombre del producto
    a = soup.find('span', class_='name')
    if a:
        palas.append(a.text.strip())
    else:
        palas.append("No name")

    # Extraer el precio del producto
    b = soup.select_one('span.price')
    if b:
        precio.append(b.text.strip())
    else:
        precio.append("No price")

    attrs = soup.select('span.description-attributes-value')

    # Buscar y asignar valores para cada categoría
    marca.append(encontrar_valor(attrs, 'Marca', keywords['Marca']))
    color.append(encontrar_valor(attrs, 'Color', keywords['Color']))
    producto.append(encontrar_valor(attrs, 'Producto', keywords['Producto']))
    balance.append(encontrar_valor(attrs, 'Balance', keywords['Balance']))
    nucleo.append(encontrar_valor(attrs, 'Nucleo', keywords['Nucleo']))
    cara.append(encontrar_valor(attrs, 'Cara', keywords['Cara']))
    dureza.append(encontrar_valor(attrs, 'Dureza', keywords['Dureza']))
    nivel_de_juego.append(encontrar_valor(attrs, 'Nivel de Juego', keywords['Nivel de Juego']))
    acabado.append(encontrar_valor(attrs, 'Acabado', keywords['Acabado']))
    forma.append(encontrar_valor(attrs, 'Forma', keywords['Forma']))
    superficie.append(encontrar_valor(attrs, 'Superficie', keywords['Superficie']))
    tipo_de_juego.append(encontrar_valor(attrs, 'Tipo de Juego', keywords['Tipo de Juego']))
    jugador.append(encontrar_valor(attrs, 'Jugador', keywords['Jugador']))

# Función para procesar cada página de productos
def procesar_pagina(web):
    try:
        page = requests.get(web)
        page.raise_for_status()
    except requests.RequestException as e:
        print(f"Error al acceder a {web}: {e}")
        return []

    soup = BeautifulSoup(page.content, 'html.parser')
    product_containers = soup.find_all('div', class_="product-item-info")
    product_links = []
    for container in product_containers:
        enlace_tag = container.find('a', class_='product-item-link')
        if enlace_tag:
            enlace = enlace_tag.get('href')
            if enlace:
                product_links.append(enlace)
    return product_links

# Usamos concurrent.futures para procesar las páginas de productos concurrentemente
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Primero, obtenemos todos los enlaces de productos
    futures = [executor.submit(procesar_pagina, web) for web in webs]
    product_links = []
    for future in concurrent.futures.as_completed(futures):
        product_links.extend(future.result())

# Usamos concurrent.futures para extraer los detalles de los productos concurrentemente
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(extraer_detalles_producto, link) for link in product_links]
    for future in concurrent.futures.as_completed(futures):
        future.result()

# Crear un DataFrame con los datos obtenidos
df = pd.DataFrame({
    'Palas': palas,
    'Precio': precio,
    'Marca': marca,
    'Color': color,
    'Producto': producto,
    'Balance': balance,
    'Nucleo': nucleo,
    'Cara': cara,
    'Dureza': dureza,
    'Nivel de Juego': nivel_de_juego,
    'Acabado': acabado,
    'Forma': forma,
    'Superficie': superficie,
    'Tipo de Juego': tipo_de_juego,
    'Jugador': jugador})

# Guardar el DataFrame en un archivo CSV
df.to_csv('palas_padel.csv', index=False)

print("Scraping completado y datos guardados en 'palas_padel.csv'")


Scraping completado y datos guardados en 'palas_padel.csv'


In [5]:
print (df)

                                     Palas    Precio        Marca  \
0                          HARLEM EUPHORIA   63,95 €      No data   
1                VARLION LW DIFUSOR W 2021  149,95 €      varlion   
2              DUNLOP BOOST LITE 2.0 WOMAN  109,00 €  black crown   
3                BLACK CROWN PITON 11 2023   86,95 €       dunlop   
4                       SIUX FURTIVE PRO 2   62,95 €         siux   
..                                     ...       ...          ...   
929            DUNLOP IMPACT PRO HL ORANGE   59,95 €       dunlop   
930                         VIBOR-A VIPERA   64,95 €      vibor-a   
931  STARVIE TITANIA KEPLER SPEED 2.0 2023   69,95 €      No data   
932                       J.HAYBER WARRIOR  192,95 €      No data   
933              DROP SHOT CONQUEROR 10 BT  146,95 €    drop shot   

                    Color Producto      Balance         Nucleo  \
0            negro, verde    palas  alto, medio       soft eva   
1             negro, rosa    palas     

In [6]:
# Nombre del archivo CSV
csv_file = 'palas_padel.csv'

# Nombre del archivo JSON de salida
json_file = 'palas_padel.json'

# Leer el archivo CSV
df = pd.read_csv(csv_file)

# Convertir el DataFrame a JSON
df.to_json(json_file, orient='records', lines=True)

print(f"Archivo CSV '{csv_file}' convertido a JSON y guardado como '{json_file}'")

Archivo CSV 'palas_padel.csv' convertido a JSON y guardado como 'palas_padel.json'
